# Module 08: Quality Validation and File Integrity

**Difficulty**: ⭐⭐
**Estimated Time**: 45 minutes
**Prerequisites**: [Module 00](00_music_library_manager.ipynb), [Module 01](01_metadata_management.ipynb)

## Learning Objectives

By the end of this notebook, you will be able to:

1. Detect corrupted or unreadable audio files in your library
2. Validate metadata integrity and identify encoding issues
3. Perform comprehensive library health checks
4. Automate library maintenance tasks (cleanup empty folders, fix common issues)
5. Generate detailed health and quality reports

## Why Quality Validation Matters

Over time, music libraries can develop issues:

- **Corrupted files**: Files that can't be played or read properly
- **Metadata problems**: Invalid characters, encoding issues, missing tags
- **Library clutter**: Empty folders, temporary files, orphaned data
- **Structural issues**: Inconsistent naming, broken links in playlists

This module helps you identify and fix these issues **before** they cause problems.

## What This Module Covers

1. **File Integrity Validation**
   - Verify audio files can be opened and read
   - Detect corrupted or incomplete files
   - Check file headers and metadata structures

2. **Metadata Validation**
   - Identify encoding issues (non-UTF-8 characters)
   - Find invalid or suspicious metadata values
   - Check for consistency across fields

3. **Library Health Checks**
   - Find empty folders (cleanup candidates)
   - Detect duplicate folder names
   - Identify orphaned files (outside normal structure)
   - Check for naming inconsistencies

4. **Automated Maintenance**
   - Safe cleanup of empty folders
   - Batch fix common metadata issues
   - Generate comprehensive health reports
   - Schedule regular health checks

---

## Setup

Import required libraries and configure settings.

In [ ]:
# Standard library imports
import os
import sys
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Optional, Tuple
from collections import Counter, defaultdict
import hashlib
import re

# Third-party imports
import pandas as pd
from IPython.display import display, HTML
from mutagen import File
from mutagen.id3 import ID3

In [ ]:
# Configuration
MUSIC_LIBRARY_PATH = Path('../../music').resolve()

# Supported audio file extensions
AUDIO_EXTENSIONS = {'.mp3', '.flac', '.wav', '.m4a', '.aac', '.ogg', '.wma', '.opus'}

# Print configuration
print(f"Music Library Path: {MUSIC_LIBRARY_PATH}")
print(f"Library exists: {MUSIC_LIBRARY_PATH.exists()}")
print(f"Supported formats: {', '.join(sorted(AUDIO_EXTENSIONS))}")

---

## Part 1: File Integrity Validation

Check if audio files can be properly read and played.

### Why File Integrity Matters

Corrupted files can cause:
- Media players to crash or skip
- Incomplete metadata extraction
- Data loss if not detected early

**Common causes of corruption:**
- Incomplete downloads
- Disk errors
- Failed file transfers
- Power outages during write operations

In [ ]:
def validate_audio_file(file_path: Path) -> Dict:
    """
    Validate a single audio file for integrity.
    
    Checks:
    - File can be opened by mutagen
    - File has valid audio properties
    - File size is reasonable (not 0 bytes or suspiciously small)
    
    Returns:
        Dictionary with validation results and any error messages
    """
    result = {
        'file': str(file_path.relative_to(MUSIC_LIBRARY_PATH)),
        'valid': True,
        'errors': [],
        'warnings': [],
        'size_bytes': 0,
        'can_read': False,
        'has_audio_properties': False
    }
    
    try:
        # Check file size
        stat_info = file_path.stat()
        result['size_bytes'] = stat_info.st_size
        
        # Suspiciously small file (less than 1KB)
        if stat_info.st_size < 1024:
            result['errors'].append(f"File too small: {stat_info.st_size} bytes")
            result['valid'] = False
        
        # Suspiciously small for audio (less than 10KB)
        elif stat_info.st_size < 10240:
            result['warnings'].append(f"Suspiciously small: {stat_info.st_size / 1024:.1f} KB")
        
        # Try to open with mutagen
        audio = File(file_path)
        
        if audio is None:
            result['errors'].append("Cannot read file: mutagen returned None")
            result['valid'] = False
        else:
            result['can_read'] = True
            
            # Check for audio properties (bitrate, length, etc.)
            if hasattr(audio, 'info') and audio.info is not None:
                result['has_audio_properties'] = True
                
                # Check for zero-length audio
                if hasattr(audio.info, 'length') and audio.info.length == 0:
                    result['errors'].append("Audio has zero duration")
                    result['valid'] = False
                    
                # Check for invalid bitrate
                if hasattr(audio.info, 'bitrate') and audio.info.bitrate == 0:
                    result['warnings'].append("Bitrate is 0 (may be variable bitrate)")
            else:
                result['errors'].append("No audio properties found")
                result['valid'] = False
    
    except PermissionError:
        result['errors'].append("Permission denied")
        result['valid'] = False
    except Exception as e:
        result['errors'].append(f"Exception: {type(e).__name__}: {str(e)}")
        result['valid'] = False
    
    return result

In [ ]:
def scan_library_for_validation(library_path: Path) -> List[Dict]:
    """
    Scan entire library and validate each audio file.
    
    This can take several minutes for large libraries.
    Progress is shown every 100 files.
    """
    print(f"Scanning library: {library_path}")
    print("Validating file integrity...\n")
    
    validation_results = []
    file_count = 0
    
    for root, dirs, files in os.walk(library_path):
        for file in files:
            file_path = Path(root) / file
            
            if file_path.suffix.lower() in AUDIO_EXTENSIONS:
                result = validate_audio_file(file_path)
                validation_results.append(result)
                file_count += 1
                
                # Show progress
                if file_count % 100 == 0:
                    print(f"Validated {file_count} files...")
    
    print(f"\n✓ Validation complete: {file_count} files scanned")
    return validation_results

In [ ]:
def get_validation_summary(validation_results: List[Dict]) -> Dict:
    """
    Summarize validation results.
    """
    total_files = len(validation_results)
    valid_files = sum(1 for r in validation_results if r['valid'])
    invalid_files = total_files - valid_files
    
    files_with_warnings = sum(1 for r in validation_results if r['warnings'])
    files_with_errors = sum(1 for r in validation_results if r['errors'])
    
    # Collect all error types
    error_types = Counter()
    for r in validation_results:
        for error in r['errors']:
            # Extract error type (first part before colon)
            error_type = error.split(':')[0]
            error_types[error_type] += 1
    
    return {
        'total_files': total_files,
        'valid_files': valid_files,
        'invalid_files': invalid_files,
        'files_with_warnings': files_with_warnings,
        'files_with_errors': files_with_errors,
        'health_percentage': (valid_files / total_files * 100) if total_files > 0 else 0,
        'error_types': dict(error_types)
    }


def print_validation_summary(summary: Dict):
    """
    Print a formatted validation summary.
    """
    print("=" * 60)
    print("FILE INTEGRITY VALIDATION SUMMARY")
    print("=" * 60)
    print(f"Total files scanned:     {summary['total_files']:,}")
    print(f"Valid files:             {summary['valid_files']:,} ({summary['health_percentage']:.1f}%)")
    print(f"Invalid files:           {summary['invalid_files']:,}")
    print(f"Files with warnings:     {summary['files_with_warnings']:,}")
    print(f"Files with errors:       {summary['files_with_errors']:,}")
    
    if summary['error_types']:
        print("\nError Types:")
        for error_type, count in sorted(summary['error_types'].items(), key=lambda x: x[1], reverse=True):
            print(f"  - {error_type}: {count}")
    
    print("=" * 60)

### 🎯 Exercise 1: Validate Your Library

Scan your library and check for corrupted files.

In [ ]:
# Scan and validate all files
validation_results = scan_library_for_validation(MUSIC_LIBRARY_PATH)

# Get summary
summary = get_validation_summary(validation_results)
print_validation_summary(summary)

In [ ]:
# View invalid files
invalid_files = [r for r in validation_results if not r['valid']]

if invalid_files:
    print(f"\n⚠️  Found {len(invalid_files)} invalid file(s):\n")
    df = pd.DataFrame(invalid_files)
    display(df[['file', 'errors', 'size_bytes']])
else:
    print("\n✓ All files are valid!")

In [ ]:
# View files with warnings
warned_files = [r for r in validation_results if r['warnings']]

if warned_files:
    print(f"\n⚠️  Found {len(warned_files)} file(s) with warnings:\n")
    df = pd.DataFrame(warned_files)
    display(df[['file', 'warnings', 'size_bytes']])
else:
    print("\n✓ No warnings!")

---

## Part 2: Metadata Validation

Check for metadata quality issues like encoding problems and invalid values.

### Common Metadata Issues

- **Encoding problems**: Non-UTF-8 characters that display as �����
- **Invalid values**: Impossible years (e.g., 0000, 9999)
- **Inconsistencies**: Artist name spelled differently across files
- **Special characters**: Characters that break file systems or players

In [ ]:
def validate_metadata(file_path: Path) -> Dict:
    """
    Validate metadata quality for a single file.
    
    Checks for:
    - Encoding issues (non-printable characters)
    - Invalid year values
    - Suspiciously long field values
    - Problematic characters in metadata
    """
    result = {
        'file': str(file_path.relative_to(MUSIC_LIBRARY_PATH)),
        'metadata_valid': True,
        'issues': [],
        'field_issues': {}
    }
    
    try:
        audio = File(file_path, easy=True)
        
        if audio is None:
            result['issues'].append("Cannot read metadata")
            result['metadata_valid'] = False
            return result
        
        # Check each metadata field
        for tag_name in ['artist', 'album', 'title', 'genre', 'date']:
            if tag_name in audio:
                value = audio[tag_name]
                if isinstance(value, list):
                    value = str(value[0]) if value else ''
                else:
                    value = str(value)
                
                field_issues = []
                
                # Check for encoding issues (non-printable or replacement characters)
                if '�' in value or any(ord(c) < 32 and c not in '\n\r\t' for c in value):
                    field_issues.append("Possible encoding issue")
                    result['metadata_valid'] = False
                
                # Check for suspiciously long values
                if len(value) > 200:
                    field_issues.append(f"Unusually long ({len(value)} chars)")
                
                # Special checks for year/date
                if tag_name == 'date':
                    year_match = re.search(r'\b(\d{4})\b', value)
                    if year_match:
                        year = int(year_match.group(1))
                        current_year = datetime.now().year
                        if year < 1900 or year > current_year + 1:
                            field_issues.append(f"Invalid year: {year}")
                            result['metadata_valid'] = False
                
                # Check for problematic characters in artist/album/title
                if tag_name in ['artist', 'album', 'title']:
                    # Characters that cause issues in file systems
                    problematic_chars = set('/\\:*?"<>|')
                    found_chars = problematic_chars & set(value)
                    if found_chars:
                        field_issues.append(f"Problematic chars: {''.join(found_chars)}")
                
                if field_issues:
                    result['field_issues'][tag_name] = {
                        'value': value[:100],  # Truncate for display
                        'issues': field_issues
                    }
        
        if result['field_issues']:
            result['issues'].append(f"{len(result['field_issues'])} field(s) have issues")
    
    except Exception as e:
        result['issues'].append(f"Exception: {str(e)}")
        result['metadata_valid'] = False
    
    return result

In [ ]:
def scan_library_for_metadata_validation(library_path: Path) -> List[Dict]:
    """
    Scan library and validate metadata for all audio files.
    """
    print(f"Scanning library: {library_path}")
    print("Validating metadata quality...\n")
    
    metadata_results = []
    file_count = 0
    
    for root, dirs, files in os.walk(library_path):
        for file in files:
            file_path = Path(root) / file
            
            if file_path.suffix.lower() in AUDIO_EXTENSIONS:
                result = validate_metadata(file_path)
                metadata_results.append(result)
                file_count += 1
                
                if file_count % 100 == 0:
                    print(f"Validated {file_count} files...")
    
    print(f"\n✓ Metadata validation complete: {file_count} files scanned")
    return metadata_results

### 🎯 Exercise 2: Check Metadata Quality

In [ ]:
# Scan for metadata issues
metadata_results = scan_library_for_metadata_validation(MUSIC_LIBRARY_PATH)

# Find files with metadata issues
files_with_issues = [r for r in metadata_results if not r['metadata_valid'] or r['field_issues']]

print(f"\nFiles with metadata issues: {len(files_with_issues)} / {len(metadata_results)}")

if files_with_issues:
    df = pd.DataFrame(files_with_issues)
    display(df[['file', 'issues']].head(20))
else:
    print("✓ All metadata is valid!")

---

## Part 3: Library Health Checks

Check for structural issues in your library organization.

### What We Check

1. **Empty folders**: Folders with no audio files (cleanup candidates)
2. **Orphaned files**: Files in the root directory (should be in folders)
3. **Duplicate folder names**: Multiple folders with same name in different locations
4. **Hidden files**: Temporary or system files that shouldn't be there

In [ ]:
def find_empty_folders(library_path: Path) -> List[Dict]:
    """
    Find all folders that contain no audio files.
    
    These are candidates for cleanup.
    """
    empty_folders = []
    
    for root, dirs, files in os.walk(library_path, topdown=False):
        folder_path = Path(root)
        
        # Skip the root library folder itself
        if folder_path == library_path:
            continue
        
        # Count audio files in this folder (not subfolders)
        audio_files = [f for f in files if Path(f).suffix.lower() in AUDIO_EXTENSIONS]
        
        # If no audio files and no subdirectories, it's empty
        subdirs = [d for d in (folder_path / d for d in dirs) if d.exists()]
        
        if len(audio_files) == 0 and len(subdirs) == 0:
            empty_folders.append({
                'path': str(folder_path.relative_to(library_path)),
                'absolute_path': str(folder_path),
                'other_files': len(files),  # Non-audio files
                'parent': str(folder_path.parent.relative_to(library_path))
            })
    
    return empty_folders

In [ ]:
def find_orphaned_files(library_path: Path) -> List[Dict]:
    """
    Find audio files in the root directory (not in any subfolder).
    
    These should typically be organized into artist/album folders.
    """
    orphaned_files = []
    
    # Only check immediate children of library_path, not recursive
    for item in library_path.iterdir():
        if item.is_file() and item.suffix.lower() in AUDIO_EXTENSIONS:
            stat_info = item.stat()
            orphaned_files.append({
                'filename': item.name,
                'path': str(item.relative_to(library_path)),
                'size_mb': stat_info.st_size / (1024 * 1024),
                'modified': datetime.fromtimestamp(stat_info.st_mtime).strftime('%Y-%m-%d')
            })
    
    return orphaned_files

In [ ]:
def find_duplicate_folder_names(library_path: Path) -> List[Dict]:
    """
    Find folders with the same name in different locations.
    
    This might indicate:
    - Duplicate artists/albums
    - Organizational inconsistencies
    - Files that should be merged
    """
    folder_names = defaultdict(list)
    
    for root, dirs, files in os.walk(library_path):
        for dir_name in dirs:
            folder_path = Path(root) / dir_name
            folder_names[dir_name.lower()].append({
                'name': dir_name,
                'path': str(folder_path.relative_to(library_path)),
                'parent': str(folder_path.parent.relative_to(library_path))
            })
    
    # Find names that appear more than once
    duplicates = []
    for name, locations in folder_names.items():
        if len(locations) > 1:
            duplicates.append({
                'folder_name': locations[0]['name'],
                'count': len(locations),
                'locations': [loc['path'] for loc in locations]
            })
    
    return sorted(duplicates, key=lambda x: x['count'], reverse=True)

In [ ]:
def generate_health_report(library_path: Path) -> Dict:
    """
    Generate a comprehensive library health report.
    """
    print("Generating library health report...\n")
    
    report = {
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'library_path': str(library_path)
    }
    
    # Find issues
    print("Checking for empty folders...")
    report['empty_folders'] = find_empty_folders(library_path)
    
    print("Checking for orphaned files...")
    report['orphaned_files'] = find_orphaned_files(library_path)
    
    print("Checking for duplicate folder names...")
    report['duplicate_folders'] = find_duplicate_folder_names(library_path)
    
    print("\n✓ Health report complete")
    return report

### 🎯 Exercise 3: Library Health Check

In [ ]:
# Generate health report
health_report = generate_health_report(MUSIC_LIBRARY_PATH)

print("\n" + "=" * 60)
print("LIBRARY HEALTH REPORT")
print("=" * 60)
print(f"Generated: {health_report['timestamp']}")
print(f"Library: {health_report['library_path']}")
print()
print(f"Empty folders found:     {len(health_report['empty_folders'])}")
print(f"Orphaned files found:    {len(health_report['orphaned_files'])}")
print(f"Duplicate folder names:  {len(health_report['duplicate_folders'])}")
print("=" * 60)

In [ ]:
# View empty folders
if health_report['empty_folders']:
    print(f"\nEmpty folders ({len(health_report['empty_folders'])}):")
    df = pd.DataFrame(health_report['empty_folders'])
    display(df[['path', 'other_files']])
else:
    print("\n✓ No empty folders found")

In [ ]:
# View orphaned files
if health_report['orphaned_files']:
    print(f"\nOrphaned files in root directory ({len(health_report['orphaned_files'])}):")
    df = pd.DataFrame(health_report['orphaned_files'])
    display(df)
else:
    print("\n✓ No orphaned files in root directory")

In [ ]:
# View duplicate folder names
if health_report['duplicate_folders']:
    print(f"\nDuplicate folder names ({len(health_report['duplicate_folders'])}):")
    for dup in health_report['duplicate_folders'][:10]:  # Show top 10
        print(f"\n'{dup['folder_name']}' appears {dup['count']} times:")
        for location in dup['locations']:
            print(f"  - {location}")
else:
    print("\n✓ No duplicate folder names found")

---

## Part 4: Automated Maintenance

Tools to automatically fix common issues.

### Safe Cleanup Operations

All maintenance operations support **dry-run mode** to preview changes before applying them.

In [ ]:
def cleanup_empty_folders(library_path: Path, dry_run: bool = True) -> List[Dict]:
    """
    Remove empty folders from the library.
    
    Args:
        library_path: Root library path
        dry_run: If True, only preview changes without deleting
    
    Returns:
        List of folders that were (or would be) deleted
    """
    empty_folders = find_empty_folders(library_path)
    actions = []
    
    if dry_run:
        print(f"DRY RUN: Would delete {len(empty_folders)} empty folder(s)")
    else:
        print(f"Deleting {len(empty_folders)} empty folder(s)...")
    
    for folder_info in empty_folders:
        folder_path = Path(folder_info['absolute_path'])
        
        action = {
            'folder': folder_info['path'],
            'action': 'DELETE' if not dry_run else 'WOULD DELETE',
            'success': False
        }
        
        if not dry_run:
            try:
                folder_path.rmdir()  # Only works on empty directories
                action['success'] = True
            except Exception as e:
                action['error'] = str(e)
        else:
            action['success'] = True  # Would succeed in dry run
        
        actions.append(action)
    
    if not dry_run:
        successful = sum(1 for a in actions if a['success'])
        print(f"✓ Deleted {successful} / {len(empty_folders)} folders")
    
    return actions

In [ ]:
def generate_full_health_report_text(library_path: Path, 
                                     validation_results: Optional[List[Dict]] = None,
                                     metadata_results: Optional[List[Dict]] = None,
                                     health_report: Optional[Dict] = None) -> str:
    """
    Generate a comprehensive text report combining all validation results.
    
    This can be saved to a file for record-keeping.
    """
    lines = []
    lines.append("=" * 70)
    lines.append("MUSIC LIBRARY HEALTH REPORT")
    lines.append("=" * 70)
    lines.append(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    lines.append(f"Library: {library_path}")
    lines.append("")
    
    # File integrity section
    if validation_results:
        lines.append("FILE INTEGRITY")
        lines.append("-" * 70)
        summary = get_validation_summary(validation_results)
        lines.append(f"Total files:        {summary['total_files']:,}")
        lines.append(f"Valid files:        {summary['valid_files']:,} ({summary['health_percentage']:.1f}%)")
        lines.append(f"Invalid files:      {summary['invalid_files']:,}")
        lines.append(f"Files with errors:  {summary['files_with_errors']:,}")
        lines.append("")
    
    # Metadata quality section
    if metadata_results:
        lines.append("METADATA QUALITY")
        lines.append("-" * 70)
        total = len(metadata_results)
        valid = sum(1 for r in metadata_results if r['metadata_valid'])
        lines.append(f"Total files:           {total:,}")
        lines.append(f"Valid metadata:        {valid:,} ({valid/total*100:.1f}%)")
        lines.append(f"Metadata issues:       {total - valid:,}")
        lines.append("")
    
    # Library health section
    if health_report:
        lines.append("LIBRARY ORGANIZATION")
        lines.append("-" * 70)
        lines.append(f"Empty folders:         {len(health_report['empty_folders'])}")
        lines.append(f"Orphaned files:        {len(health_report['orphaned_files'])}")
        lines.append(f"Duplicate folder names: {len(health_report['duplicate_folders'])}")
        lines.append("")
    
    lines.append("=" * 70)
    lines.append("END OF REPORT")
    lines.append("=" * 70)
    
    return "\n".join(lines)

### 🎯 Exercise 4: Cleanup Empty Folders

In [ ]:
# Preview cleanup (dry run)
cleanup_actions = cleanup_empty_folders(MUSIC_LIBRARY_PATH, dry_run=True)

if cleanup_actions:
    print("\nFolders that would be deleted:")
    df = pd.DataFrame(cleanup_actions)
    display(df[['folder', 'action']])
else:
    print("\n✓ No empty folders to clean up")

In [ ]:
# Uncomment to actually delete empty folders
# WARNING: This will permanently delete folders!

# cleanup_actions = cleanup_empty_folders(MUSIC_LIBRARY_PATH, dry_run=False)
# print("Cleanup complete!")

### 🎯 Exercise 5: Generate Complete Health Report

In [ ]:
# Generate comprehensive report
full_report_text = generate_full_health_report_text(
    MUSIC_LIBRARY_PATH,
    validation_results=validation_results if 'validation_results' in locals() else None,
    metadata_results=metadata_results if 'metadata_results' in locals() else None,
    health_report=health_report if 'health_report' in locals() else None
)

print(full_report_text)

In [ ]:
# Save report to file
report_filename = f"library_health_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
report_path = MUSIC_LIBRARY_PATH / report_filename

# Uncomment to save report
# with open(report_path, 'w', encoding='utf-8') as f:
#     f.write(full_report_text)
# print(f"\n✓ Report saved to: {report_path}")

---

## Summary

In this module, you learned:

### Key Concepts

1. **File Integrity Validation**
   - Detect corrupted audio files
   - Check file headers and properties
   - Identify files that can't be played

2. **Metadata Validation**
   - Find encoding issues (�� characters)
   - Detect invalid metadata values
   - Check for problematic characters

3. **Library Health Checks**
   - Find empty folders
   - Detect orphaned files
   - Identify duplicate folder names

4. **Automated Maintenance**
   - Safe cleanup with dry-run mode
   - Generate comprehensive health reports
   - Schedule regular maintenance

### Best Practices

- **Run validation regularly** (monthly or after adding many new files)
- **Always use dry-run first** before any cleanup operations
- **Keep health reports** to track library quality over time
- **Fix corruption early** before files become completely unrecoverable
- **Backup before maintenance** operations that delete files

### What's Next?

- **[Module 03: Album Art Management](03_album_art_management.ipynb)** - Extract and manage album artwork
- **[Module 05: External API Integration](05_external_api_integration.ipynb)** - Enhance metadata with MusicBrainz

### Additional Resources

- **Mutagen Documentation**: [https://mutagen.readthedocs.io/](https://mutagen.readthedocs.io/)
- **Audio File Repair Tools**: Consider using tools like `mp3val` (MP3) or `flac -t` (FLAC) for detailed validation
- **Backup Strategies**: Use tools like `rsync` or cloud storage for regular backups

---

**Great job!** You now have the tools to maintain a healthy, high-quality music library. 🎵